# HYPA-DBGNN vs. GCN vs. DBGNN

## Data Sets

The data sets used in the paper are in the data folder.
Specify the number of classes as defined in the paper.

In [49]:
path_data = "data/synthetic_paths.ngram"
label_data = "data/synthetic_labels.txt"
splits_data = "data/synthetic_splits.json"
classes = 2

# Have a look at the data folder for the remaining data sets. Classes are given in the paper.

## Setup Environment

Use the devcontainer to install the required dependencies.
The following imports are needed to run the code.

In [50]:
import csv
import json
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch.nn import ModuleList, Module, Linear
from torch.nn.functional import one_hot
from torch_geometric.nn import GCNConv, MessagePassing
from torch_geometric.data import Data
from torch_geometric.transforms import RemoveDuplicatedEdges

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import balanced_accuracy_score

from tqdm import trange
import pathpyG as pp

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## Import Data

### Creation of De Bruijn graphs

Path data and labels are read and converted to higher-order De Bruijn graphs.

In [51]:
def map_labels_to_y(labels, num_nodes, node_index_to_id):
    classes = sorted(set(labels.values()))
    
    y = torch.zeros((num_nodes,), dtype=torch.int32)
    y[:] = -1

    for index in range(num_nodes):
        y[index] = classes.index(labels[node_index_to_id[index]])

    assert not -1 in y, "not all nodes have labels"

    return y, classes


def load_labels_from_file(path, num_nodes, node_index_to_id, separator = "\t", label_index = -1):
    labels = {} 
    with open(path,"r") as f:
        for line in f:
            line = line.replace("\n","")
            list_line = line.split(separator)
            labels[list_line[0]] = list_line[label_index]
    
    return map_labels_to_y(labels, num_nodes, node_index_to_id)
    

def load_labels_from_csv(path, num_nodes, node_index_to_id):
    with open(path, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        labels = {}
        for row in reader:
            labels[row[0]] = int(row[1])
            
    return map_labels_to_y(labels, num_nodes, node_index_to_id)

def load_paths(path: str) -> pp.PathData:
    return pp.PathData.from_csv(path)


def build_graph(paths: pp.PathData, k = 1) -> Data:
    graph = pp.HigherOrderGraph(paths, order=k, node_id=paths.node_id)
    data = graph.to_pyg_data()
    data.edge_index = data.edge_index.to(torch.int64)
    transform = RemoveDuplicatedEdges(key=["edge_weight"], reduce="add")
    data = transform(data)
    data.node_id_to_index = graph.node_id_to_index
    data.node_index_to_id = graph.node_index_to_id
    data.order = k
    return data

### Calculation of HYPA Scores

The HYPA Scores are calculated for the given higher-order graph.

In [52]:
_hypa_is_enabled = False
hyge = None
lcdf = None

def lhygecdf(k : torch.tensor, s : torch.tensor, f : torch.tensor, n : torch.tensor) ->  torch.tensor:
    global _hypa_is_enabled
    global hyge
    global lcdf

    if not _hypa_is_enabled:
        print("enabling hypa...", end='')
 
        from julia.api import Julia
        jl = Julia(compiled_modules=False)
        from julia.Distributions import Hypergeometric, logcdf
        
        hyge = Hypergeometric
        lcdf = logcdf

        _hypa_is_enabled = True
        print(" Done")
    
    hyge_inst = hyge(int(s.item()), int(f.item()), int(n.item()))
    return torch.tensor(lcdf(hyge_inst, k.item()))

def calculate_hypa_scores(num_nodes: int, edge_index: torch.Tensor, edge_weights: torch.Tensor, tolerance: float = 1e-5, verbose: bool = False, redistribute=True) -> tuple:
    """
    assumption: no duplicated edges
    to ensure, use:
    transform = RemoveDuplicatedEdges(key=["edge_weight"], reduce="add")
    data = transform(data)
    """

    def weighted_in_deg(edge_weights):
        return torch.zeros((num_nodes,), dtype=edge_weights.dtype, device=edge_weights.device).scatter_add_(dim=0, index=edge_index[1, :], src=edge_weights)

    def weighted_out_deg(edge_weights):
        return torch.zeros((num_nodes,), dtype=edge_weights.dtype, device=edge_weights.device).scatter_add_(dim=0, index=edge_index[0, :], src=edge_weights)

    f_v_out = weighted_out_deg(edge_weights)
    f_v_in  = weighted_in_deg(edge_weights)
    
    m = f_v_out.sum()
    M = torch.matmul(f_v_out.unsqueeze(1), f_v_in.unsqueeze(0)).sum()
    
    xi_vw = f_v_out[edge_index[0, :]] * f_v_in[edge_index[1, :]]
    
    if redistribute:
        for i in range(5000):
            # expectation for in-degrees    
            f_hat_v_in = weighted_in_deg(xi_vw) / m * M / xi_vw.sum()
            # correction of in-degrees
            xi_vw = xi_vw * (f_v_in / f_hat_v_in)[edge_index[1, :]].nan_to_num(1)
            # expectation for out-degrees
            f_hat_v_out = weighted_out_deg(xi_vw) / m * M / xi_vw.sum()
            # correction for out-degrees
            xi_vw = xi_vw * (f_v_out / f_hat_v_out)[edge_index[0, :]].nan_to_num(1)

            rmse = ((f_v_out - f_hat_v_out)**2).sum().sqrt().item() + ((f_v_in - f_hat_v_in)**2).sum().sqrt().item()
            
            if verbose:
                print(rmse)
            
            if  rmse < tolerance:
                break
        if verbose:
            print(f"Optimized for {i} iterations.")
    
    pval = torch.zeros_like(xi_vw)
    for i in range(len(xi_vw)):
        pval[i] = lhygecdf(edge_weights[i], m, xi_vw.sum() - m, xi_vw[i]).exp()

    return pval, xi_vw

### Complete Import of Data

We import the path data, build the De Bruijn graph and calculate the HYPA scores.

In [53]:
def build_data(paths: str, labels: str, max_order: int = 1, verbose: bool = False, redistribute=True) -> Data:
    assert max_order >= 1

    if verbose:
        print('Reading paths...')
    path_data = load_paths(paths)

    graph_data = []
    for k in range(1, max_order+1):
        if verbose:
            print(f'Building graph data with order {k}...')
        data = build_graph(path_data, k = k)

        if verbose:
            print(f'--- Calculating Hypa Scores for order {k}...')
        data.pval, data.xi = calculate_hypa_scores(data.num_nodes, data.edge_index, data.edge_weight, redistribute=redistribute, verbose=verbose)

        graph_data.append(data)

        
    if verbose:
        print('Reading labels ...')
    if labels.endswith('.txt'):
        y, classes = load_labels_from_file(labels, graph_data[0].num_nodes, graph_data[0].node_index_to_id)
    else:
        y, classes = load_labels_from_csv(labels, graph_data[0].num_nodes, graph_data[0].node_index_to_id)

    edge_index = [data.edge_index for data in graph_data]
    edge_weight = [data.edge_weight for data in graph_data]
    edge_attr = [data.pval for data in graph_data]
    edge_xi = [data.xi for data in graph_data]
    
    num_nodes = [data.num_nodes for data in graph_data]
    
    node_idx_to_id = [data.node_index_to_id for data in graph_data]
    node_id_to_idx = [data.node_id_to_index for data in graph_data]

    order = [data.order for data in graph_data]

    def build_x_one_hot(num_nodes):
        return one_hot(torch.arange(num_nodes), num_classes = num_nodes).to(torch.float)
    
    x = build_x_one_hot(num_nodes[0]) 
    x_h = build_x_one_hot(num_nodes[-1])    

    def build_bipartite_mapping_low_to_high(id2idx_low, idx2id_high):
        return torch.tensor([[id2idx_low[idx2id_high[idx_high][0]], idx_high] for idx_high in idx2id_high.keys()], dtype=torch.long).t()


    def build_bipartite_mapping_high_to_low(id2idx_low, idx2id_high, pos=-1):
        return torch.tensor([[idx_high, id2idx_low[idx2id_high[idx_high][pos]]] for idx_high in idx2id_high.keys()], dtype=torch.long).t()

    data = Data(
        edge_index                  = edge_index[0],
        edge_index_higher_order     = edge_index[-1],
        bipartite_edge_index        = build_bipartite_mapping_high_to_low(node_id_to_idx[0], node_idx_to_id[-1], pos=0),
        bipartite_edge_index_start  = build_bipartite_mapping_low_to_high(node_id_to_idx[0], node_idx_to_id[-1]),
        edge_weight                 = edge_weight[0],
        edge_weight_higher_order    = edge_weight[-1],
        x                           = x,
        x_h                         = x_h,
        y                           = y.long(),
        num_nodes                   = num_nodes[0],
        num_ho_nodes                = num_nodes[-1],
        # hypa scores:
        edge_attr                   = edge_attr[0].unsqueeze(1),
        edge_attr_higher_order      = edge_attr[-1].unsqueeze(1),
        node_id_to_idx              = node_id_to_idx,
        node_idx_to_id              = node_idx_to_id,
    )
    return data

## Models

### GCN (Kipf & Welling)

In [54]:
"""
Graph convolutional network as proposed in:
'Semi-Supervised Classification with Graph Convolutional Networks'
Paper: https://arxiv.org/abs/1609.02907
Website: https://tkipf.github.io/graph-convolutional-networks/
"""

class GCN(Module):
    def __init__(
        self,
        num_classes,
        num_features,
        hidden_dims,
        p_dropout=0.0
        ):
        super().__init__()

        self.num_features = num_features
        self.num_classes = num_classes
        self.hidden_dims = hidden_dims
        self.p_dropout = p_dropout

        # first-order layers
        self.first_order_layers = ModuleList()
        self.first_order_layers.append(GCNConv(self.num_features, self.hidden_dims[0]))

        for dim in range(1, len(self.hidden_dims)):
            # first-order layers
            self.first_order_layers.append(GCNConv(self.hidden_dims[dim-1], self.hidden_dims[dim]))

        # Linear layer
        self.lin = torch.nn.Linear(self.hidden_dims[-1], num_classes)



    def forward(self, data):

        x = data.x

        # First-order convolutions
        for layer in self.first_order_layers:
            x = F.dropout(x, p=self.p_dropout, training=self.training)
            x = F.elu(layer(x, data.edge_index, data.edge_weight))
        x = F.dropout(x, p=self.p_dropout, training=self.training)

        # Linear layer
        x = self.lin(x)

        return x

### DBGNN

In [55]:
"""
DBGNN model as proposed in 
'De Bruijn goes Neural: Causality-Aware Graph Neural Networks for Time Series Data on Dynamic Graphs'
Paper: https://arxiv.org/abs/2209.08311
Presentation: https://www.youtube.com/watch?v=IezbzMMp9QM
"""

class DBGNNBipartiteGraphOperator(MessagePassing):
    def __init__(self, in_ch, out_ch):
        super(DBGNNBipartiteGraphOperator, self).__init__('add')
        self.lin1 = Linear(in_ch, out_ch)
        self.lin2 = Linear(in_ch, out_ch)

    def forward(self, x, bipartite_index, N, M):
        x = (self.lin1(x[0]), self.lin2(x[1]))
        return self.propagate(bipartite_index, size=(N, M), x=x)

    def message(self, x_i, x_j):
        return x_i + x_j

class DBGNN(Module):
    def __init__(
        self,
        num_classes,
        num_features,
        hidden_dims,
        p_dropout=0.0
        ):
        super().__init__()

        self.num_features = num_features
        self.num_classes = num_classes
        self.hidden_dims = hidden_dims
        self.p_dropout = p_dropout

        # higher-order layers
        self.higher_order_layers = ModuleList()
        self.higher_order_layers.append(GCNConv(self.num_features[1], self.hidden_dims[0]))

        # first-order layers
        self.first_order_layers = ModuleList()
        self.first_order_layers.append(GCNConv(self.num_features[0], self.hidden_dims[0]))

        for dim in range(1, len(self.hidden_dims)-1):
            # higher-order layers
            self.higher_order_layers.append(GCNConv(self.hidden_dims[dim-1], self.hidden_dims[dim]))
            # first-order layers
            self.first_order_layers.append(GCNConv(self.hidden_dims[dim-1], self.hidden_dims[dim]))

        self.bipartite_layer = DBGNNBipartiteGraphOperator(self.hidden_dims[-2], self.hidden_dims[-1])

        # Linear layer
        self.lin = torch.nn.Linear(self.hidden_dims[-1], num_classes)



    def forward(self, data):

        x = data.x
        x_h = data.x_h

        # First-order convolutions
        for layer in self.first_order_layers:
            x = F.dropout(x, p=self.p_dropout, training=self.training)
            x = F.elu(layer(x, data.edge_index, data.edge_weight))
        x = F.dropout(x, p=self.p_dropout, training=self.training)

        # Second-order convolutions
        for layer in self.higher_order_layers:
            x_h = F.dropout(x_h, p=self.p_dropout, training=self.training)
            x_h = F.elu(layer(x_h, data.edge_index_higher_order, data.edge_weight_higher_order))
        x_h = F.dropout(x_h, p=self.p_dropout, training=self.training)

        # Bipartite message passing
        x = torch.nn.functional.elu(self.bipartite_layer((x_h, x), data.bipartite_edge_index, x_h.size(0), x.size(0)))
        x = F.dropout(x, p=self.p_dropout, training=self.training)

        # Linear layer
        x = self.lin(x)

        return x

### HYPA-DBGNN

In [56]:
from torch_geometric.nn import MessagePassing
import torch

class BipartiteGraphOperator(MessagePassing):
    def __init__(self, in_ch_source, in_ch_target, out_ch):
        super(BipartiteGraphOperator, self).__init__('add')
        self.lin_source = Linear(in_ch_source, out_ch)
        self.lin_target = Linear(in_ch_target, out_ch) if in_ch_target > 0 else None

    def forward(self, x_source, x_target, bipartite_index):
        x_source = self.lin_source(x_source)
        x_target = torch.zeros((bipartite_index[1, :].max()+1, x_source.size(1))).to(x_source.device) if self.lin_target is None else self.lin_target(x_target)
        return self.propagate(bipartite_index, size=(x_source.size(0), x_target.size(0)), x=(x_source, x_target))

    def message(self, x_i, x_j):
        return x_i + x_j
    

class HYPA_DBGNN(Module): 
    def __init__(
        self,
        num_classes,
        num_features,
        num_edge_features,
        hidden_dims,
        p_dropout=0.0
        ):
        super().__init__()

        self.num_features = num_features
        self.num_edge_features = num_edge_features
        self.num_classes = num_classes
        self.hidden_dims = hidden_dims
        self.p_dropout = p_dropout

        assert len(num_features) == 1
        self.bipartite_layer_start = BipartiteGraphOperator(self.num_features[0], 0, self.num_features[0])

        # higher-order layers
        self.higher_order_layers = ModuleList()
        self.higher_order_layers.append(GCNConv(self.num_features[0], self.hidden_dims[0]))

        # first-order layers
        self.first_order_layers = ModuleList()
        self.first_order_layers.append(GCNConv(self.num_features[0], self.hidden_dims[0]))


        for dim in range(1, len(self.hidden_dims)-1):
            # higher-order layers
            self.higher_order_layers.append(GCNConv(self.hidden_dims[dim-1], self.hidden_dims[dim]))
            # first-order layers
            self.first_order_layers.append(GCNConv(self.hidden_dims[dim-1], self.hidden_dims[dim]))

        self.bipartite_layer = BipartiteGraphOperator(self.hidden_dims[-2], self.hidden_dims[-2], self.hidden_dims[-1])

        # Linear layer
        self.lin = torch.nn.Linear(self.hidden_dims[-1], num_classes)



    def forward(self, data):

        x = data.x
        x_ = F.dropout(x, p=self.p_dropout, training=self.training)
        x_h = torch.nn.functional.elu(self.bipartite_layer_start(x_, None, data.bipartite_edge_index_start))

        edge_weight = data.edge_attr 
        edge_weight_higher_order = data.edge_attr_higher_order 

        # First-order convolutions
        layer = self.first_order_layers[0]
        x = F.dropout(x, p=self.p_dropout, training=self.training)
        x = F.elu(layer(x, data.edge_index, edge_weight=edge_weight))

        for layer in self.first_order_layers[1:]:
            x = F.dropout(x, p=self.p_dropout, training=self.training)
            x = F.elu(layer(x, data.edge_index, edge_weight=edge_weight))
        x = F.dropout(x, p=self.p_dropout, training=self.training)

        # Second-order convolutions
        layer = self.higher_order_layers[0]
        x_h = F.dropout(x_h, p=self.p_dropout, training=self.training)
        x_h = F.elu(layer(x_h, data.edge_index_higher_order, edge_weight=edge_weight_higher_order))

        for layer in self.higher_order_layers[1:]:
            x_h = F.dropout(x_h, p=self.p_dropout, training=self.training)
            x_h = F.elu(layer(x_h, data.edge_index_higher_order, edge_weight=edge_weight_higher_order))
        x_h = F.dropout(x_h, p=self.p_dropout, training=self.training)

        # Bipartite message passing
        x = torch.nn.functional.elu(self.bipartite_layer(x_h, x, data.bipartite_edge_index))
        x = F.dropout(x, p=self.p_dropout, training=self.training)

        # Linear layer
        x = self.lin(x)

        return x

## Evaluation Procedure

We use a 10 fold cross-validation and select the best epoch with a validation split.
Test performance is reported afterwards. 

In [57]:
def evaluate(data, splits, build_model, epochs, seed):

    results = []

    for fold in range(10):
    
        # reset seeds used in training
        torch.manual_seed(seed)
        random.seed(seed)
        np.random.seed(seed)

        # calculate weights for loss function
        def calculate_class_weights(y: torch.Tensor):
            class_weights = y.to(torch.float).unique(sorted=True, return_counts=True)[1]
            num_elements = class_weights.sum()
            return 1.0-class_weights/num_elements
        class_weights = calculate_class_weights(data.y).to(device)

        # define loss function for training
        loss_fn = CrossEntropyLoss(class_weights)

        model = build_model()
        optimizer = SGD(model.parameters(), lr=0.001, weight_decay=0.0005, momentum=0.9)

        # obtain indices from splits
        if splits == None:
            split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=seed)
            for train_valid_index, test_index in split.split(np.zeros(len(data.y.cpu())), data.y.cpu()):
                break

            split2 = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=seed)
            for train_index, valid_index in split2.split(np.zeros(len(train_valid_index)), data.y[train_valid_index].cpu()):
                train_index = train_valid_index[train_index]
                valid_index = train_valid_index[valid_index]
        else:
            test_index = splits[fold]['test_index']
            valid_index = splits[fold]['valid_index']
            train_index = splits[fold]['train_index']

        best_valid_loss = 100000
        result = 0.0

        # train this trial
        for epoch in trange(epochs, position=0, desc=f"learning fold {fold+1}"):
            model.train()

            data = data.to(device)
            output = model(data)
            loss = loss_fn(output[train_index], data.y[train_index])

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            model.eval()
            output = model(data)

            valid_loss = loss_fn(output[valid_index], data.y[valid_index])
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                _, pred = output.max(dim=1)
                result = balanced_accuracy_score(data.y[test_index].cpu(), pred[test_index].cpu())

        results.append(result)

    print(f"Balanced Accuracy: {np.mean(results)} ± {np.std(results)}")

## Execute Experiments

### Load data

In [58]:
data = build_data(path_data, label_data, max_order=2, verbose=False, redistribute=True).to(device)

enabling hypa... Done


In [59]:
with open(splits_data, 'r') as file:
    splits = json.loads(file.read())['splits']

### Define Models and Run Experiments

In [60]:
def hypa_dbgnn():
    return HYPA_DBGNN(
            num_classes         = classes,
            num_features        = [data.x.size(-1)],
            num_edge_features   = [data.edge_attr.size(-1), data.edge_attr_higher_order.size(-1)],
            hidden_dims         = [32, 32, 16],
            p_dropout           = 0.4
    ).to(device)
def dbgnn():
    return DBGNN(
            num_classes         = classes,
            num_features        = [data.x.size(-1), data.x_h.size(-1)],
            hidden_dims         = [32, 32, 16],
            p_dropout           = 0.4
    ).to(device)
def gcn():
    return GCN(
            num_classes         = classes,
            num_features        = data.x.size(-1),
            hidden_dims         = [32, 32, 16],
            p_dropout           = 0.4
    ).to(device)


print("HYPA-DBGNN:")
evaluate(data, splits, hypa_dbgnn, epochs=5000, seed=0)
print("\nGCN:")
evaluate(data, splits, gcn, epochs=5000, seed=0)
print("\nDBGNN:")
evaluate(data, splits, dbgnn, epochs=5000, seed=0)

HYPA-DBGNN:


learning fold 10: 100%|██████████| 5000/5000 [05:42<00:00, 14.61it/s]


Balanced Accuracy: 1.0 ± 0.0

GCN:


learning fold 10: 100%|██████████| 5000/5000 [02:27<00:00, 33.93it/s]


Balanced Accuracy: 0.5 ± 0.31622776601683794

DBGNN:


learning fold 10: 100%|██████████| 5000/5000 [04:43<00:00, 17.64it/s]


Balanced Accuracy: 0.5 ± 0.22360679774997896
